In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
from math import sqrt, erf, ceil, pi, exp

## Function for Stage 2 Sample size

In [8]:
def sample_size(s, d2, alpha, B2):
    Za = norm.ppf(1 - alpha)
    Zb = norm.ppf(B2)
    return (2 * (s * (Za + Zb)) ** 2) / (d2 ** 2)

## Case 1

In [9]:
alpha, d1, d2 = 0.05, 0.0, 0.2
Mu0, K, Beta = [0.2,0.4], [2,3,4,5], [0.70, 0.75, 0.80,0.85,0.9]
n1_LB=1
n1_UB=200
s=1

Mu0_fin, K_fin, n1_fin, n2_fin, Gamma_fin, En0_fin, En_fin, Nmax_fin = [], [], [], [], [], [], [], []
pi0_bar_fin, beta1_fin, beta2_fin, beta_fin = [], [], [], []

Gamma = np.arange(0.0, 0.3, 0.005)  # gamma values

for m0 in Mu0:
    m1, m2 = m0 + d1, m0 + d2  
    
    for k in K:
        for B in Beta:
            En_arr_min, n1_arr, beta1_arr_fin, beta2_arr_fin = [], [], [], []
            n2_arr_fin, En0_arr_fin, En1_arr_fin, N_arr_fin, pi0_bar_arr_fin = [], [], [], [], []
            
            for g in Gamma:
                En_arr, beta1_arr, beta2_arr, n2_arr, En0_arr, En1_arr, pi0_bar_arr = [], [], [], [], [], [], []
                
                for n1 in range(n1_LB,n1_UB):
                    # Beta 1
                    def function(z):
                        f1= (1/sqrt(2*pi))*exp(-0.5*z**2)
                        
                        x=z+(sqrt(n1)*(d2-d1))/s
                        f2=(0.5*(1+erf(x/sqrt(2))))**(k-1)
                        f=f1*f2
                        return f
        
                    LB=sqrt(n1)*(g-d2)/s
                    UB=np.inf
                    beta1, error = quad(function, LB, UB)
                    
                    beta2 = B / beta1
                    beta1_arr.append(beta1)
                    beta2_arr.append(beta2)
                    
                    n2=sample_size(s,d2,alpha,beta2) if beta2 < 1 else 5000
                    n2_arr.append(n2)
                    
                    pi0=1-norm.cdf(g*sqrt(n1)/s)**k
                    pi1=1-(norm.cdf((g-d1)*sqrt(n1)/s)**(k-1))*norm.cdf((g-d2)*sqrt(n1)/s)
                    
                    pi0_bar_arr.append(1 - pi0)
                    
                    En0, En1 = k * n1 + pi0 * 2 * n2, k * n1 + pi1 * 2 * n2
                    En0_arr.append(En0)
                    En1_arr.append(En1)
                    En_arr.append(En0 * 0.5 + En1 * 0.5 if En0 > 0 else 1000)
                    
                x = np.argmin(En_arr)
                n1_arr.append(x + 1)
                En_arr_min.append(En_arr[x])
                beta1_arr_fin.append(beta1_arr[x])
                beta2_arr_fin.append(beta2_arr[x])
                n2_arr_fin.append(ceil(n2_arr[x]))
                En0_arr_fin.append(En0_arr[x])
                En1_arr_fin.append(En1_arr[x])
                pi0_bar_arr_fin.append(pi0_bar_arr[x])
            
            if 5000 in n2_arr_fin:
                Gamma = Gamma[:n2_arr_fin.index(5000)]
            
            En_arr_min, n1_arr, n2_arr_fin, En0_arr_fin, pi0_bar_arr_fin = (
                En_arr_min[:len(Gamma)], n1_arr[:len(Gamma)], n2_arr_fin[:len(Gamma)], En0_arr_fin[:len(Gamma)], pi0_bar_arr_fin[:len(Gamma)]
            )
            
            Gamma_arr = [Gamma[i] for i, en in enumerate(En_arr_min) if en == min(En_arr_min)]
            Gamma_range = [round(min(Gamma_arr), 3), round(max(Gamma_arr), 3)]
            
            x1 = np.argmin(En_arr_min)
            
            Mu0_fin.append(m0)
            K_fin.append(k)
            n1_fin.append(n1_arr[x1])
            n2_fin.append(n2_arr_fin[x1])
            Gamma_fin.append(Gamma_range)
            En0_fin.append(En0_arr_fin[x1])
            En_fin.append(En_arr_min[x1])
            Nmax_fin.append(k * n1_arr[x1] + 2 * n2_arr_fin[x1])
            pi0_bar_fin.append(pi0_bar_arr_fin[x1])
            beta1_fin.append(beta1_arr_fin[x1])
            beta2_fin.append(beta2_arr_fin[x1])
            beta_fin.append(B)

data = pd.DataFrame({
    "Theta0": Mu0_fin, "K": K_fin, "n1": n1_fin, "n2": n2_fin,
    "Gamma": Gamma_fin, "E0(N)": En0_fin, "E(N)": En_fin, "N": Nmax_fin,
    "1-pi0": pi0_bar_fin, "Beta 1": beta1_fin, "Beta 2": beta2_fin, "Beta": beta_fin
})

data

,Theta0,K,n1,n2,Gamma,E0(N),E(N),N,1-pi0,Beta 1,Beta 2,Beta
0,0.2,2,110,333,"[0.09, 0.09]",429.932276,623.524500,886,0.684585,0.848370,0.825111,0.70
1,0.2,2,129,372,"[0.09, 0.09]",468.188802,700.994375,1002,0.716829,0.872254,0.859841,0.75
2,0.2,2,148,411,"[0.085, 0.085]",524.627251,792.653414,1118,0.721562,0.899999,0.888889,0.80
3,0.2,2,173,459,"[0.08, 0.08]",594.838810,906.655430,1264,0.728726,0.927142,0.916796,0.85
4,0.2,2,199,538,"[0.075, 0.075]",687.369890,1062.609491,1474,0.730980,0.948407,0.948959,0.90
5,0.2,3,117,366,"[0.1, 0.1]",616.804304,811.916243,1083,0.636720,0.818281,0.855451,0.70
6,0.2,3,134,400,"[0.095, 0.095]",685.501806,910.210761,1202,0.645576,0.850656,0.881672,0.75
7,0.2,3,155,439,"[0.09, 0.09]",767.243093,1026.704244,1343,0.655663,0.882787,0.906221,0.80
8,0.2,3,181,489,"[0.085, 0.085]",868.748048,1171.833428,1521,0.666703,0.913241,0.930751,0.85
9,0.2,3,199,581,"[0.075, 0.075]",1032.435739,1378.733027,1759,0.624968,0.936510,0.961015,0.90


## Case 2

In [10]:
alpha, d1, d2 = 0.05, 0.0, 0.2
Mu0, K, Beta, Beta2_fixed1, Beta2_fixed2 = [0.2], [2,3,4,5], [0.70, 0.75, 0.80], [0.80, 0.85, 0.90], [0.85, 0.875, 0.90]
n1_LB=1
n1_UB=200
s=1

data_results = []

for m0 in Mu0:
    for k in K:
        for B in Beta:
            if B == Beta2_fixed1[0]:
                Beta2_fixed = Beta2_fixed2
            else:
                Beta2_fixed = Beta2_fixed1  
            
            for Beta2 in Beta2_fixed:  
                m1, m2 = m0 + d1, m0 + d2
                Gamma = np.arange(0.0, 0.15, 0.005)
                min_En, Gamma_range, optimal_values = float('inf'), [], None

                for g in Gamma:
                    for n1 in range(n1_LB,n1_UB):
                        def function(z):
                            f1= (1/sqrt(2*pi))*exp(-0.5*z**2)
                            
                            x=z+(sqrt(n1)*(d2-d1))/s
                            f2=(0.5*(1+erf(x/sqrt(2))))**(k-1)
                            f=f1*f2
                            return f
        
                        LB=sqrt(n1)*(g-d2)/s
                        UB=np.inf
                        beta1f, error = quad(function, LB, UB)

                        beta1_constraint = B / Beta2 if Beta2 != 0 else float('inf')  # Prevent division by zero
                        if beta1f >= beta1_constraint:
                            beta2 = Beta2
                            n2 = ceil(sample_size(s,d2,alpha,beta2)) if beta2 < 1 else 500
                            pi0=1-norm.cdf(g*sqrt(n1)/s)**k
                            pi1=1-(norm.cdf((g-d1)*sqrt(n1)/s)**(k-1))*norm.cdf((g-d2)*sqrt(n1)/s)
                    
                            pi0_bar_arr.append(1 - pi0)
                            En0 = k * n1 + pi0 * 2 * n2
                            En = 0.5 * En0 + 0.5 * (k * n1 + pi1 * 2 * n2)
                            N_max = k * n1 + 2 * n2

                            if En < min_En:
                                min_En = En
                                Gamma_range = [g, g]
                                optimal_values = (m0, k, n1, n2, Gamma_range, En0, min_En, N_max,1-pi0, beta1f, Beta2, B)
                            elif En == min_En:
                                Gamma_range[1] = g

                if optimal_values:
                    optimal_values = list(optimal_values)
                    optimal_values[4] = f"[{round(Gamma_range[0],3)}, {round(Gamma_range[1],3)}]"
                    data_results.append(tuple(optimal_values))

data = pd.DataFrame(data_results, columns=["Mu0", "K", "n1", "n2", "Gamma", "E(N_0)", "E(N)", "N_max", "1-pi0", "Beta1", "Beta2", "Beta"])
data

,Mu0,K,n1,n2,Gamma,E(N_0),E(N),N_max,1-pi0,Beta1,Beta2,Beta
0,0.2,2,118,310,"[0.08, 0.08]",451.642442,629.738416,856,0.652190,0.875356,0.800,0.70
1,0.2,2,106,360,"[0.1, 0.1]",413.765832,626.578733,932,0.719770,0.823538,0.850,0.70
2,0.2,2,107,429,"[0.12, 0.12]",388.170856,651.967195,1072,0.797004,0.778288,0.900,0.70
3,0.2,2,153,310,"[0.055, 0.055]",575.530481,742.271512,926,0.565273,0.937707,0.800,0.75
4,0.2,2,131,360,"[0.085, 0.085]",480.369582,702.924559,982,0.696709,0.882889,0.850,0.75
5,0.2,2,122,429,"[0.105, 0.105]",442.196685,716.789942,1102,0.769002,0.834071,0.900,0.75
6,0.2,2,170,360,"[0.065, 0.065]",597.307942,817.344607,1060,0.642628,0.941316,0.850,0.80
7,0.2,2,149,391,"[0.075, 0.075]",554.140603,796.701316,1080,0.672454,0.914848,0.875,0.80
8,0.2,2,145,429,"[0.09, 0.09]",512.300047,795.935277,1148,0.740909,0.889001,0.900,0.80
9,0.2,3,137,310,"[0.08, 0.08]",682.277413,839.724426,1031,0.562456,0.875624,0.800,0.70
